# Cell Tracking and Lineage Construction in Live-Cell Imaging Data
---


### Global Imports

In [1]:
import os
import datetime
import errno
import argparse

import numpy as np

import deepcell

Using TensorFlow backend.


### Load the Training Data

In [2]:
# Download the data (saves to ~/.keras/datasets)
filename = '3T3_NIH.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.nih_3t3.load_tracked_data(filename)

print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

X.shape: (188, 30, 154, 182, 1)
y.shape: (188, 30, 154, 182, 1)


### Set up filepath constants

In [3]:
# The path to the data file is currently required for `train_model_()` functions

# Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
DATA_DIR = "/data/npz_data/cells/HEK293/generic/movie/"                # USE LOCAL DATA INSTEAD

# DATA_FILE should be a trks file (contains 2 np arrays and a lineage dictionary)
DATA_FILE = os.path.join(DATA_DIR, filename)
DATA_FILE = os.path.join(DATA_DIR, '3T3_HeLa_HEK_corrected.trks')      # USE LOCAL DATA INSTEAD

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [4]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Set up training parameters

In [5]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

tracking_model_name = 'tracking_model'

n_epoch = 5  # Number of training epochs
test_size = .10  # % of data saved as test

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Tracking training settings
features = {"appearance", "distance", "neighborhood", "regionprop"}
min_track_length = 5
neighborhood_scale_size=30
batch_size = 128  

in_shape = (32, 32, 1) # Should this be calculated or hardcoded?
seed = 100 # To be removed

### Training the Model

#### Instantiate the tracking model

In [6]:
from deepcell import model_zoo

tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

#### Option 1: Train a new tracking model

In [7]:
from deepcell.training import train_model_siamese_daughter

tracking_model = train_model_siamese_daughter(
    model=tracking_model,
    dataset=DATA_FILE,  # full path to trks file
    model_name=tracking_model_name,
    optimizer=optimizer,
    batch_size=batch_size,
    min_track_length=min_track_length,
    features=features,
    neighborhood_scale_size=neighborhood_scale_size,
    n_epoch=n_epoch,
    model_dir=MODEL_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    class_weight=None,
    seed = seed)

training on dataset: /data/npz_data/cells/HEK293/generic/movie/3T3_HeLa_HEK_corrected.trks
saving model at: /data/models/tracking_model.h5
saving loss at: /data/models/tracking_model.npz
X_train shape: (555, 40, 216, 256, 1)
y_train shape: (555, 40, 216, 256, 1)
X_test shape: (62, 40, 216, 256, 1)
y_test shape: (62, 40, 216, 256, 1)
Output Shape: (None, 3)
Training on 1 GPUs
Using real-time data augmentation.


/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabl

total_train_pairs: 6649520.0
total_test_pairs: 653280.0
batch size: 128
validation_steps:  5103.0
 29]
 24 25 26 27 28 29]
 24 25 26 27 28 29]
 24]
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
 24 25 26 27 28 29]
Epoch 1/5


StopIteration: zero-size array to reduction operation minimum which has no identity

In [ ]:
1 / 0

#### Option 2: Load an existing tracking model

In [ ]:
# Import the tracking model
MODEL_DIR = '/data/models/'
PREFIX = 'cells/3T3/NIH/'

# Re-instantiate the model and load weights
siamese_weights_file = '2019-01-18_nuclear_movie_3T3_and_HeLa_complete_[a,d,n,r]_neighs=30_epochs=5_seed=101_trks_0.h5'
siamese_weights_file = os.path.join(MODEL_DIR, PREFIX, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

#### Verify Model Accuracy with Confusion Matrix

In [ ]:
import deepcell.image_generators as generators

datagen_val = generators.SiameseDataGenerator(
        rotation_range=0,  # randomly rotate images by 0 to rotation_range degrees
        shear_range=0,     # randomly shear images in the range (radians , -shear_range to shear_range)
        horizontal_flip=0, # randomly flip images
        vertical_flip=0)   # randomly flip images

val_iterator = generators.SiameseIterator((x_test, y_test),
                                          datagen_val,
                                          neighborhood_scale_size=neighborhood_scale_size,
                                          min_track_length=min_track_length,
                                          features=features)

In [ ]:
from sklearn.metrics import confusion_matrix

Y = []
Y_pred = []
for i in range(1,1001):
    if i % 100 == 0:
        print(".", end="")
    lst, y_true = next(val_iterator)
    y_true = list(map(np.argmax, y_true))
    y_pred = list(map(np.argmax, tracking_model.predict(lst)))
    Y.extend(y_true)
    Y_pred.extend(y_pred)

confusion_matrix(Y, Y_pred)

In [ ]:
val_acc = sum(np.array(Y) == np.array(Y_pred)) / len(Y)
print("Accuracy across all three classes: ", val_acc)

### Track Multiple Movies and Generate Track Files

#### Run the model

In [ ]:
# Normalize raw images if needed
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

for batch in range(data_readable_X.shape[0]):
    for frame in range(data_readable_X.shape[1]):
        data_readable_X[batch, frame, :, :, 0] = image_norm(data_readable_X[batch, frame, :, :, 0]) 


In [ ]:
# Define where cell tracks will be saved
base_path = "/data/data/cells/3T3/NIH/set0/deepcell_segmentations_v1/part_2/tracks/"
base_name = "3T3_S0P2_Batch"

# Generate a cell track for each batch
for batch in range (data['X'].shape[0]):
    trial = deepcell.tracking.cell_tracker(data['X'][batch], data['y'][batch],
                         tracking_model,
                         max_distance=200,
                         track_length=5, division=0.5, birth=0.9, death=0.9,
                         neighborhood_scale_size=30,
                         features=features)
    trial._track_cells()
    file_name = base_name + str(batch).zfill(2) + '.trk'
    file_path = os.path.join(base_path, file_name)
    trial.dump(file_path)

### Review the results

In [ ]:

ax[5].imshow(watershed_images[index, frame, ..., 0], cmap='jet')


# Can also export as a video
# But this does not render well on GitHub
from IPython.display import HTML
from deepcell.utils.plot_utils import get_js_video

HTML(get_js_video(watershed_images[..., [-1]], batch=index))

#### Save the Raw and Tracked Output

In [ ]:
channel = 0

for i in range(45):
    #name_raw = os.paHeLa_and_3T3th.join('tracking_movies/test_true2_{:02}_.png'.format(i))
    name_tracked = os.path.join('tracking_movies/test_tracked3_{:02}_.png'.format(i))
    #plt.imsave(name_raw, test_data['y'][0, i, :, :, channel], cmap='jet')
    plt.imsave(name_tracked, trial.y_tracked[i, :, :, channel], cmap='cubehelix', vmin=0, vmax=250)
